In [37]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

In [38]:
from f1tenth_gym_jax import make
from f1tenth_gym_jax.envs.utils import Param
import jax
import numpy as np

In [39]:
num_agents = 5
num_envs = 100

In [40]:
env = make(f"Spielberg_{num_agents}_noscan_time_v0")

In [41]:
rng = jax.random.PRNGKey(0)
rng, _rng = jax.random.split(rng)
reset_rng = jax.random.split(_rng, num_envs)

In [42]:
obsv, env_state = jax.vmap(env.reset)(reset_rng)

In [43]:
rng, _rng = jax.random.split(rng)
action_rng = jax.random.split(_rng, num_envs*num_agents).reshape((num_agents, num_envs, -1))
actions = {k: jax.vmap(env.action_space(k).sample)(action_rng[i]) for i, k in enumerate(env.agents)}

In [44]:
rng, _rng = jax.random.split(rng)
step_rng = jax.random.split(_rng, num_envs)
obsv, env_state, _, _, info = jax.vmap(env.step)(step_rng, env_state, actions)

In [45]:
def env_init(rng):
    rng, _rng = jax.random.split(rng)
    reset_rng = jax.random.split(_rng, num_envs)
    obsv, env_state = jax.vmap(env.reset)(reset_rng)
    return (env_state, obsv, rng)

def env_step(runner_state, unused):
    env_state, last_obs, rng = runner_state

    rng, _rng = jax.random.split(rng)
    action_rng = jax.random.split(_rng, num_envs*num_agents).reshape((num_agents, num_envs, -1))
    actions = {k: jax.vmap(env.action_space(k).sample)(action_rng[i]) for i, k in enumerate(env.agents)}
    for k in env.agents:
        actions[k] = actions[k].at[:, 0].set(0.0)
        actions[k] = actions[k].at[:, 1].set(2.0)

    rng, _rng = jax.random.split(rng)
    step_rng = jax.random.split(_rng, num_envs)
    obsv, env_state, _, _, info = jax.vmap(env.step)(step_rng, env_state, actions)

    runner_state = (env_state, obsv, rng)
    return runner_state, runner_state

In [52]:
actions['agent_0'].shape

(100, 2)

In [46]:
runner_state = env_init(rng)
last_runner_state, all_runner_states = jax.lax.scan(env_step, runner_state, length=50)

In [47]:
all_runner_states[0].cartesian_states.shape

(50, 100, 5, 7)

In [48]:
import importlib
# import f1tenth_gym_jax
import f1tenth_gym_jax.envs.rendering.renderer as r
importlib.reload(r)

<module 'f1tenth_gym_jax.envs.rendering.renderer' from '/home/billyzheng/f1tenth_gym_jax/f1tenth_gym_jax/envs/rendering/renderer.py'>

In [49]:
player = r.TrajRenderer(env)

In [50]:
try:
    player.render(np.array(all_runner_states[0].cartesian_states))
except Exception as e:
    print(e)
    player.close()

Trajectory to render: 1
Trajectory to render: 2
Trajectory to render: 3
Trajectory to render: 2


/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:641: RuntimeWarning: overflow encountered in cast
  if maxRng[ax] is not None and diff > maxRng[ax]:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:654: RuntimeWarning: overflow encountered in cast
  if lmn is not None and mn < lmn:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:658: RuntimeWarning: overflow encountered in cast
  elif lmx is not None and mx > lmx:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1640: RuntimeWarning: overflow encountered in cast
  if maxRng[axis] is not None and maxRng[axis] < range:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1643: RuntimeWarning: overflow encountered in cast
  if limits[axis][0

Trajectory to render: 3


/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:641: RuntimeWarning: overflow encountered in cast
  if maxRng[ax] is not None and diff > maxRng[ax]:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:654: RuntimeWarning: overflow encountered in cast
  if lmn is not None and mn < lmn:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:658: RuntimeWarning: overflow encountered in cast
  elif lmx is not None and mx > lmx:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1640: RuntimeWarning: overflow encountered in cast
  if maxRng[axis] is not None and maxRng[axis] < range:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1643: RuntimeWarning: overflow encountered in cast
  if limits[axis][0

Exception: Cannot set range [nan, nan]

...

Trajectory to render: 4


/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:641: RuntimeWarning: overflow encountered in cast
  if maxRng[ax] is not None and diff > maxRng[ax]:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:654: RuntimeWarning: overflow encountered in cast
  if lmn is not None and mn < lmn:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:658: RuntimeWarning: overflow encountered in cast
  elif lmx is not None and mx > lmx:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1640: RuntimeWarning: overflow encountered in cast
  if maxRng[axis] is not None and maxRng[axis] < range:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1643: RuntimeWarning: overflow encountered in cast
  if limits[axis][0

Trajectory to render: 5


/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:641: RuntimeWarning: overflow encountered in cast
  if maxRng[ax] is not None and diff > maxRng[ax]:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:654: RuntimeWarning: overflow encountered in cast
  if lmn is not None and mn < lmn:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:658: RuntimeWarning: overflow encountered in cast
  elif lmx is not None and mx > lmx:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1640: RuntimeWarning: overflow encountered in cast
  if maxRng[axis] is not None and maxRng[axis] < range:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1643: RuntimeWarning: overflow encountered in cast
  if limits[axis][0

play pause
